# 第5回AIエッジコンテストの学習用動画データ(mp4)を読みこみ、真値を矩形で描画して動画(mp4)で保存する。

In [14]:
import cv2
import json
from PIL import Image
from PIL import ImageColor

In [15]:
# 各種パスの設定
train_movie_path = "./train_videos/train_00.mp4"
train_annotations_path = "./train_annotations/train_00.json"
categories_path = "./docs/categories" # カテゴリファイル。1行ごとにカテゴリ名が記載。

In [16]:
# 学習用動画データのアノテーションのロード
with open(train_annotations_path, 'r') as f:
    train_annotations_json = json.load(f)
train_annotations_json['sequence'][0]

{'Signal': [{'id': 56, 'box2d': [655, 194, 754, 236]},
  {'id': 57, 'box2d': [858, 443, 893, 455]}],
 'Signs': [{'id': 149, 'box2d': [440, 433, 494, 483]}],
 'Pedestrian': [{'id': 8972, 'box2d': [1239, 568, 1290, 755]},
  {'id': 9004, 'box2d': [1748, 617, 1789, 713]},
  {'id': 8985, 'box2d': [531, 563, 552, 618]},
  {'id': 8984, 'box2d': [446, 558, 494, 684]},
  {'id': 8980, 'box2d': [821, 560, 952, 784]},
  {'id': 8989, 'box2d': [326, 552, 360, 653]},
  {'id': 8995, 'box2d': [1337, 595, 1353, 635]},
  {'id': 8996, 'box2d': [353, 554, 417, 686]},
  {'id': 8997, 'box2d': [1512, 591, 1533, 648]},
  {'id': 8992, 'box2d': [281, 546, 321, 650]},
  {'id': 9000, 'box2d': [1582, 594, 1615, 684]},
  {'id': 9001, 'box2d': [1103, 591, 1113, 618]},
  {'id': 9002, 'box2d': [1045, 590, 1055, 615]},
  {'id': 9003, 'box2d': [1356, 592, 1368, 629]},
  {'id': 8920, 'box2d': [1695, 604, 1737, 722]}],
 'Car': [{'id': 1177, 'box2d': [1194, 601, 1243, 622]}],
 'Motorbike': [{'id': 70, 'box2d': [1598, 622, 1

In [17]:
# 学習用動画データのロード
video = cv2.VideoCapture(train_movie_path)
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
size = (width, height)
frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
frame_rate = int(video.get(cv2.CAP_PROP_FPS))

print("画像の(横,縦)サイズ：" + str(size))
print("画像の枚数：" + str(frame_count))
print("フレームレート(fps)：" + str(frame_rate))

画像の(横,縦)サイズ：(1936, 1216)
画像の枚数：600
フレームレート(fps)：5


In [23]:
class CategoriesReader():
    categories = []
    categories_color = {}
    def __init__(self, categories_file_path):
        with open(categories_file_path, 'r') as f:
            categories = f.readlines()
            self.categories = [c.strip() for c in categories] # 改行文字の削除
        self.assign_colors()
    def get_categories(self):
        return self.categories

    def assign_colors(self):
        '''
        HSVの色相(H)をカテゴリ数で分割し、カテゴリごとに色を割り当てる
        '''
        cat_num = len(self.categories)
        for i, category in enumerate(self.categories):
            h = int(360 / (cat_num)) * i
            s = 100 # 100%
            v = 100 # 100%
            color = ImageColor.getrgb("hsv({},{}%,{}%)".format(str(h), str(s), str(v)))
            self.categories_color[category] = color

    def get_category_color(self, category):
        return self.categories_color[category]

# カテゴリファイルの読み込みを行う
categories = CategoriesReader(categories_path)
category_names = categories.get_categories()
print(category_names)

# 矩形描画用にカテゴリごとに色を割り当てられているのを確認する
for category in category_names:
    print("{}: RGB is {}".format(category , str(categories.get_category_color(category))))

['Car', 'Bus', 'Truck', 'Svehicle', 'Pedestrian', 'Motorbike', 'Bicycle', 'Train', 'Signal', 'Signs']
Car: RGB is (255, 0, 0)
Bus: RGB is (255, 153, 0)
Truck: RGB is (204, 255, 0)
Svehicle: RGB is (51, 255, 0)
Pedestrian: RGB is (0, 255, 102)
Motorbike: RGB is (0, 255, 255)
Bicycle: RGB is (0, 102, 255)
Train: RGB is (51, 0, 255)
Signal: RGB is (204, 0, 255)
Signs: RGB is (255, 0, 153)


In [11]:
# 一つの真値の座標をとりだしてみる
# 動画の最初のフレームの、信号器とされている領域の1つ目のデータについて、
# そのidと領域を表す矩形の情報[left, top, right, bottom]を取得する。
train_annotations_json['sequence'][0]['Signal'][0]

{'id': 56, 'box2d': [655, 194, 754, 236]}

In [25]:
# シーケンス番号
sequence = 0

# 動画保存用にVideoWriterオブジェクト生成
fmt = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
writer = cv2.VideoWriter('./result.mp4', fmt, frame_rate, size)

while True:
    ret, frame = video.read()
    if ret is False:
        break
    # Jsonファイルからシーケンス番号に関連するカテゴリを読み込む
    for category in category_names:
        category_values = train_annotations_json['sequence'][sequence].get(category)
        if category_values is not None:
            # 矩形を検出したら
            for rect_info in category_values:
                id_number = rect_info['id']
                box2d = rect_info['box2d']
                # 画像に矩形を描画する
                pt1 = (box2d[0], box2d[1])
                pt2 = (box2d[2], box2d[3])
                color = categories.get_category_color(category)
                cv2.rectangle(frame, pt1, pt2, color, thickness=5)
                # 矩形の上にカテゴリとid番号を記述する
                # 画像，テキスト，位置（左下），フォント，スケール，色，線太さ，種類
                cv2.putText(frame, category, pt1, cv2.FONT_HERSHEY_PLAIN, 1.5, color, 1, cv2.LINE_AA)
    writer.write(frame)
    sequence += 1
writer.release()

In [7]:
video.release()
cv2.destroyAllWindows()